In [5]:
import pandas as pd
import json

### DDT

In [6]:
ddt_df = pd.read_excel('../../../datasets/new_platform/ddt/ddt_comp_with_sim.xlsx', index_col=0)
ddt_df = ddt_df.drop(columns=['sim_indic_1'	,'sim_indic_2',	'sim_indic_3'])


In [7]:

ddt_df['on_page'] = 'Data Discovery Tool'
ddt_df['on_viz'] = 'Data Discovery Tool'
ddt_df.head(2)

,iso3_country_code,indicator_year,indicator_value,indicator_ddt_name,indicator_source,indicator_ddt_cat,country_name,region_name,units,on_page,on_viz
0,DZA,1995,520.0,Total International Arrivals,UNWTO,Inbound Tourism-Arrivals,Algeria,Northern Africa,Thousands,Data Discovery Tool,Data Discovery Tool
1,BEN,1995,580.0,Total International Arrivals,UNWTO,Inbound Tourism-Arrivals,Benin,Western Africa,Thousands,Data Discovery Tool,Data Discovery Tool


In [8]:
count_region = ddt_df[['country_name','iso3_country_code', 	'region_name']].drop_duplicates().reset_index().drop(columns=['index'])

### Country Profiles

In [9]:
econ_df = pd.read_excel('../../../datasets/new_platform/country_profiles/country_profiles.xlsx', sheet_name=0, index_col=0)
infra_df = pd.read_excel('../../../datasets/new_platform/country_profiles/country_profiles.xlsx',  sheet_name=1, index_col=0)
#--------
econ_df.rename(columns={'Country' :'iso3_country_code' , 'Metric' : 'indicator_ddt_name', 'Latest Year': 'indicator_year', 'Latest Value' : 'indicator_value'}, inplace=True)
econ_df = econ_df.merge(count_region, on='iso3_country_code', how='left')
econ_df['indicator_source'] = 'World Bank'
econ_df['indicator_ddt_cat'] = ''
econ_df['units'] = ''
econ_df['on_page'] = 'Country Profiles'
econ_df['on_viz'] = 'Economy'
#------

infra_df.rename(columns={'Country' :'iso3_country_code' , 'Metric' : 'indicator_ddt_name', 'Latest Year': 'indicator_year', 'Latest Value' : 'indicator_value'}, inplace=True)
infra_df = infra_df.merge(count_region, on='iso3_country_code', how='left')
infra_df['indicator_source'] = 'World Bank'
infra_df['indicator_ddt_cat'] = ''
infra_df['units'] = ''
infra_df['on_page'] = 'Country Profiles'
infra_df['on_viz'] = 'Infrastructure'

#### Un Sites
Keep Cultural as separate excel file due to different structure and static nature. 

### Bubble Dataset

#### Inbound Expenditure Percentage

In [10]:
inbdound_expend_2011 = ddt_df.query(""" indicator_year == 2012 &  indicator_ddt_name == 'Total Expenditure on Inbound Tourism' & indicator_source == 'UNWTO' """).copy()
inbdound_expend_2021 = ddt_df.query(""" indicator_year == 2022 &  indicator_ddt_name == 'Total Expenditure on Inbound Tourism' & indicator_source == 'UNWTO' """).copy()
inbdound_expend_2021.rename(columns={'indicator_value' : 'expend_inbound_total_2022'}, inplace=True)
inbdound_expend_2011.rename(columns={'indicator_value' : 'expend_inbound_total_2011'}, inplace=True)

inbdound_expend_2021 = inbdound_expend_2021.merge(inbdound_expend_2011[['iso3_country_code', 'expend_inbound_total_2011']], on=['iso3_country_code'])
inbdound_expend_2021['indicator_value'] = (inbdound_expend_2021['expend_inbound_total_2022'].astype(float) / inbdound_expend_2021['expend_inbound_total_2011'].astype(float)) * 100


In [11]:
inbdound_expend_2021 = inbdound_expend_2021[ddt_df.columns.to_list()]

In [12]:
inbdound_expend_2021.loc[:,'units'] = 'Percentage'
inbdound_expend_2021.loc[:,'on_page'] = 'Rising Tides'
inbdound_expend_2021.loc[:,'on_viz'] = 'Bubbles'
inbdound_expend_2021.loc[:,'indicator_ddt_name'] =  'Growth in Total Expenditure on Inbound Tourism 2022 (as a percentage of 2012)'
inbdound_expend_2021.loc[:,'indicator_ddt_cat'] = ''

#### Inbound Arrivals Percentage

In [13]:
inbdound_arr_2011 = ddt_df.query(""" indicator_year == 2012 &  indicator_ddt_name == 'Total International Arrivals' & indicator_source == 'UNWTO' """).copy()
inbdound_arr_2021 = ddt_df.query(""" indicator_year == 2022 &  indicator_ddt_name ==  'Total International Arrivals' & indicator_source == 'UNWTO' """).copy()
inbdound_arr_2021.rename(columns={'indicator_value' : 'arr_inbound_total_2022'}, inplace=True)
inbdound_arr_2011.rename(columns={'indicator_value' : 'earr_inbound_total_2012'}, inplace=True)

inbdound_arr_2021 = inbdound_arr_2021.merge(inbdound_arr_2011[['iso3_country_code', 'earr_inbound_total_2012']], on=['iso3_country_code'])
inbdound_arr_2021['indicator_value'] = (inbdound_arr_2021['arr_inbound_total_2022'].astype(float) / inbdound_arr_2021['earr_inbound_total_2012'].astype(float)) * 100


In [14]:
inbdound_arr_2021 = inbdound_arr_2021[ddt_df.columns.to_list()]

In [15]:
inbdound_arr_2021.loc[:,'units'] = 'Percentage'
inbdound_arr_2021.loc[:,'on_page'] = 'Rising Tides'
inbdound_arr_2021.loc[:,'on_viz'] = 'Bubbles'
inbdound_arr_2021.loc[:,'indicator_ddt_name'] =  'Growth in International Arrivals 2022 (as a percentage of 2012)'
inbdound_arr_2021.loc[:,'indicator_ddt_cat'] = ''

#### Continent

In [16]:
df_cont = ddt_df[ ddt_df['indicator_ddt_name'].isin([       'Total Travelers Originating From Africa',
       'Total Travelers Originating From Americas',
       'Total Travelers Originating From East Asia and the Pacific',
       'Total Travelers Originating From Europe',
       'Total Travelers Originating From Middle East',
       'Total Travelers Originating From Other not classified',
       'Total Travelers Originating From of which, nationals residing abroad',
       'Total Travelers Originating From South Asia'])]
df_cont = df_cont.query(""" indicator_year == 2021""")
df_cont_pivot = df_cont.pivot_table(index=['iso3_country_code','country_name','indicator_year', 'region_name'],
                                     columns='indicator_ddt_name', 
                                     values='indicator_value').reset_index()
df_cont_pivot = df_cont_pivot.reset_index().drop(columns=['iso3_country_code','country_name','indicator_year', 'region_name', 'index'])
cont_2021_thousands = pd.DataFrame(df_cont_pivot.sum()).reset_index().rename(columns={0 : 'indicator_value'})

In [17]:
for i, row in cont_2021_thousands.iterrows():
    s = row['indicator_ddt_name']
    cont_2021_thousands.loc[i,'indicator_ddt_name'] = f'{s} (Sum - 2021)'
cont_2021_thousands

,indicator_ddt_name,indicator_value
0,Total Travelers Originating From Africa (Sum -...,9278.751
1,Total Travelers Originating From Americas (Sum...,1215.174
2,Total Travelers Originating From East Asia and...,452.524
3,Total Travelers Originating From Europe (Sum -...,9298.240
4,Total Travelers Originating From Middle East (...,2659.916
5,Total Travelers Originating From Other not cla...,4375.228
6,Total Travelers Originating From South Asia (S...,324.956
7,"Total Travelers Originating From of which, nat...",4064.869


In [18]:
cont_2021_thousands.loc[:,'units'] = 'Thousands'
cont_2021_thousands.loc[:,'on_page'] = 'Rising Tides'
cont_2021_thousands.loc[:,'on_viz'] = 'Bubbles'
cont_2021_thousands.loc[:,'indicator_ddt_cat'] = ''
cont_2021_thousands.loc[:,'region_name'] = ''
cont_2021_thousands.loc[:,'iso3_country_code'] = ''
cont_2021_thousands.loc[:,'country_name'] = ''
cont_2021_thousands.loc[:,'indicator_year'] = 2021
cont_2021_thousands.loc[:,'indicator_source'] = 'UNWTO'


#### Tourism Industries

In [19]:
num_estab_2011 = ddt_df.query(""" indicator_year == 2012 &  indicator_ddt_name == 'Total Number of Establishments' & indicator_source == 'UNWTO' """).copy()
num_estab_2021 = ddt_df.query(""" indicator_year == 2022 &  indicator_ddt_name ==  'Total Number of Establishments' & indicator_source == 'UNWTO' """).copy()
num_estab_2021.rename(columns={'indicator_value' : 'num_estab_2021'}, inplace=True)
num_estab_2011.rename(columns={'indicator_value' : 'num_estab_2012'}, inplace=True)

num_estab_2021 = num_estab_2021.merge(num_estab_2011[['iso3_country_code', 'num_estab_2012']], on=['iso3_country_code'])
num_estab_2021['indicator_value'] = (num_estab_2021['num_estab_2021'].astype(float) / num_estab_2021['num_estab_2012'].astype(float)) * 100



In [20]:
num_estab_2021 = num_estab_2021[ddt_df.columns.to_list()]

In [21]:
num_estab_2021.loc[:,'units'] = 'Percentage'
num_estab_2021.loc[:,'on_page'] = 'Rising Tides'
num_estab_2021.loc[:,'on_viz'] = 'Bubbles'
num_estab_2021.loc[:,'indicator_ddt_name'] =  'Growth in Number of Tourism Establishments 2022 (as a percentage of 2012)'
num_estab_2021.loc[:,'indicator_ddt_cat'] = ''

#### Pop

In [22]:
df_pop= pd.read_excel(r'../../../datasets/dashboards/country_cultural.xlsx', sheet_name=5)
df_pop = df_pop[['iso3', 'population']]
df_pop.rename(columns={'iso3' : 'iso3_country_code'}, inplace=True)

employ_2022 = ddt_df.query(""" indicator_year == 2021 &  indicator_ddt_name == 'Total (Employment)' & indicator_source == 'UNWTO' """).copy()
employ_2022['employment'] = employ_2022['indicator_value'] * 1000
employ_2022 = employ_2022.merge(df_pop, on='iso3_country_code', how='left')
employ_2022['indicator_value'] = (employ_2022['employment'].astype(float) / employ_2022['population'].astype(float)) * 100

employ_2022 = employ_2022[ddt_df.columns.to_list()]

employ_2022.loc[:,'units'] = 'Percentage'
employ_2022.loc[:,'on_page'] = 'Rising Tides'
employ_2022.loc[:,'on_viz'] = 'Bubbles'
employ_2022.loc[:,'indicator_ddt_name'] =  'Percentage of the Population Employed in Tourism'
employ_2022.loc[:,'indicator_ddt_cat'] = ''

In [23]:
employ_2022 = employ_2022.drop_duplicates()


### Business

In [24]:
bus_indic = pd.read_excel('../../../datasets/indicators/bus_indx.xlsx', sheet_name=0, index_col=0)

In [27]:
bus_indic['Business Cluster Group'] = pd.qcut(bus_indic['Composite Index'], q=4, labels=[4, 3, 2, 1])


In [29]:
bus_indic = bus_indic.drop(columns=[col for col in bus_indic.columns if col.startswith('Fact')])
bus_indic = bus_indic.rename(columns={'Composite Index' : 'Business Connectivity Index (GBCI)', 'Rank' : 'Business Connectivity Index (GBCI) Rank'})


In [30]:
bus_indic = pd.melt(
    bus_indic, 
    id_vars=['iso3_country_code', 'country_name', 'region_name'],  # Columns to keep
    var_name='indicator_ddt_name',  # The new column name for previous column headers
    value_name='indicator_value'  # The new column name for values
)

In [31]:
bus_indic.loc[:,'units'] = ''
bus_indic.loc[:,'on_page'] = 'Rising Tides'
bus_indic.loc[:,'on_viz'] = 'Bubbles'
bus_indic.loc[:,'indicator_ddt_cat'] = ''
bus_indic.loc[:,'indicator_year'] = 2024
bus_indic.loc[:,'indicator_source'] = 'With Africa'


### Cluster

In [32]:
cluster = pd.read_excel('../../../datasets/new_platform/bubble/cluster_analysis.xlsx', index_col=0)

In [33]:
cluster.rename(columns={'Cluster' : 'Tourism Performance Classification', 'Tourism Category' : 'Tourism Performance Category'},inplace=True)


In [34]:
cluster = pd.melt(
    cluster, 
    id_vars=['iso3_country_code', 'country_name', 'region_name'],  # Columns to keep
    var_name='indicator_ddt_name',  # The new column name for previous column headers
    value_name='indicator_value'  # The new column name for values
)

In [35]:
cluster.loc[:,'units'] = ''
cluster.loc[:,'on_page'] = 'Rising Tides'
cluster.loc[:,'on_viz'] = 'Bubbles'
cluster.loc[:,'indicator_ddt_cat'] = ''
cluster.loc[:,'indicator_year'] = 2024
cluster.loc[:,'indicator_source'] = ''


### Missing
1. Country Profiles - Cultural
2. Ease of Access Dashboard
3. About our Data

In [36]:
wa_complete_db = pd.concat([ddt_df, econ_df, infra_df,inbdound_expend_2021, 
                             inbdound_arr_2021, cont_2021_thousands, num_estab_2021,
                             employ_2022, bus_indic, cluster
                            ])

In [37]:
wa_complete_db

,iso3_country_code,indicator_year,indicator_value,indicator_ddt_name,indicator_source,indicator_ddt_cat,country_name,region_name,units,on_page,on_viz
0,DZA,1995.0,520.0,Total International Arrivals,UNWTO,Inbound Tourism-Arrivals,Algeria,Northern Africa,Thousands,Data Discovery Tool,Data Discovery Tool
1,BEN,1995.0,580.0,Total International Arrivals,UNWTO,Inbound Tourism-Arrivals,Benin,Western Africa,Thousands,Data Discovery Tool,Data Discovery Tool
2,BWA,1995.0,636.0,Total International Arrivals,UNWTO,Inbound Tourism-Arrivals,Botswana,Southern Africa,Thousands,Data Discovery Tool,Data Discovery Tool
3,TCD,1995.0,58.0,Total International Arrivals,UNWTO,Inbound Tourism-Arrivals,Chad,Central Africa,Thousands,Data Discovery Tool,Data Discovery Tool
4,COD,1995.0,85.0,Total International Arrivals,UNWTO,Inbound Tourism-Arrivals,Congo (Democratic Republic of the),Central Africa,Thousands,Data Discovery Tool,Data Discovery Tool
...,...,...,...,...,...,...,...,...,...,...,...
487,TGO,2024.0,Emerging Tourism Industry,Tourism Performance Category,,,Togo,Western Africa,,Rising Tides,Bubbles
488,TUN,2024.0,Advancing Tourism Economy,Tourism Performance Category,,,Tunisia,Northern Africa,,Rising Tides,Bubbles
489,UGA,2024.0,Advancing Tourism Economy,Tourism Performance Category,,,Uganda,Eastern Africa,,Rising Tides,Bubbles
490,ZMB,2024.0,Advancing Tourism Economy,Tourism Performance Category,,,Zambia,Eastern Africa,,Rising Tides,Bubbles


In [38]:
wa_complete_db.to_excel(r'../../../datasets/new_platform/wa_complete_db.xlsx')